# 1. 學習目標

透過蒐集網路上的第三方資料並加以分析，產出洞察協助決策

# 2.  為什麼需要學習網路爬蟲

- 從研究報告、文獻中無法直接取得數據：  
  
假設你從某個研究報告中看到A電商目前的SKU為10,000個，但並不知道3C家電、服飾、美妝等各品類的SKU數，此時就能夠試著使用爬蟲來獲取這份資料

- 透過手動的方式獲取網路資料過於費時：  
  
受政府單位監管，上市上櫃的企業必須定時將其財務資訊揭露於[公開資訊觀測站](https://mops.twse.com.tw/mops/web/t57sb01_q1)，若用手動的確實可以將資料慢慢整理起來，但若希望能夠大規模自動化定時將各企業的財報給彙整再一起，就會需要使用[爬蟲](https://ithelp.ithome.com.tw/articles/10204773)來處理囉！

# 3. 整體學習的流程與架構

![img](爬蟲學習流程圖.png "test")

整個學習可以分為三個部分
- 建立爬蟲的基本概念
- 運用套件實踐爬蟲實作
- 產出分析結果

# 4. 建立爬蟲的的基本概念

## 4.1 HTML
若我們將網頁想像為一個實體建築，那HTML就是支撐網頁的骨架，而像是CSS或者是JavaScript則提供其形狀或其他支援。同時，也正因為瀏覽器能夠解讀HTML並將內容呈現給使用者，我們才能自由的將文字、圖片、影像等內容呈現在網頁上

HTML的元素可分為
- Elements :  
elements 通常都會由一個Opening tag(<>) 開始，由Closing tag(</>)，中間夾的就會是文字、圖片、連結等等的內容  

- Tag：  
Tag是HTML的主要組成元素，它使我們在HTML中的內容提供語義，並向瀏覽器表達我們的網頁該如何解釋與呈現，其用法與elements常和Tag搞混，但實際上它們是不同的，Tag指的是在element中的opening tag和closing tag  

- Attribute：  
Attribute為HTML加上了意義與功能，Attribute作為opening tag的一部分，通常在一個element中會有複數個attrubute，常用的用法為 <'tagname attributename=”setting”>content for the html element…<'tagname>  

- Content：  
各種要呈現在網頁上的內容

![image](https://i0.wp.com/www.edupointbd.com/wp-content/uploads/2017/09/elements.png?resize=608%2C353)

而透過HTML呈現的網頁結構[可以參考](https://www.w3schools.com/html/html_intro.asp)下列敘述與圖表
- The <!DOCTYPE html> declaration defines that this document is an HTML5 document
- The <!html> element is the root element of an HTML page
- The <!head> element contains meta information about the HTML page
- The <!title> element specifies a title for the HTML page (which is shown in the browser's title bar or in the page's tab)
- The <!body> element defines the document's body, and is a container for all the visible contents, such as headings, paragraphs, images, hyperlinks, tables, lists, etc.
- The <!h1> element defines a large heading
- The <!p> element defines a paragraph

![image](https://miro.medium.com/max/1050/1*H0Y0UaNr8Xu7Fxsn7Cf5iQ.png)

## 4.2 爬蟲實作

### 4.2.1 匯入需要的套件

In [ ]:
# basic
import pandas as pd
import requests #抓網頁原始碼

# bs4
from bs4 import BeautifulSoup #解析網頁原始碼

# Selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

#others
import cloudscraper

### 4.2.2 將Dcard設定為想要解析的網址

In [ ]:
url = 'https://www.dcard.tw/f' #設定dcard的網頁連結
requ = requests.get(url) #抓取網頁原始碼
print('This is requ：\n{}'.format(requ))#requ傳回的會是一個response代號
print('-'*50)
print('This is requ.text：\n{}'.format(requ.text)) #利用requ.text獲得裡面的原始碼

### 4.2.3 使用BeautifulSoup解析網頁源代碼,能夠得到一個 BeautifulSoup 的物件,並利用.prettify()來縮進格式的結構輸出以方便閱讀

In [ ]:
soup = BeautifulSoup(requ.text, 'html.parser') #利用html.parser解析原始碼
print(soup.prettify())

### 4.2.4 由於Tag向我們表達HTML不同區塊的內容，因此我們<font color="db5957"><b>第一個目標希望能夠先找到Dcard文章的超連結網址</b></font>，為了達成這個怒標，我們先透過Tag找到所有包含a標籤(tags)的節點(elements)

>HTML 超連結 <'a> 標籤 (tag)
全世界的網頁或網路資源利用連結彼此互相引用互相鏈接，形成一個大互聯網，而 HTML <'a> tag 就是用來建立超連結 (hyperlink) -- 通往其他頁面、檔案、Email 地址、或其他 URL 的超連結。

In [ ]:
a_tags = soup.find_all('a')
print(a_tags)

### 4.2.5 從上面的輸出結果可以看到，我們需要的超連結網址是在href的content當中，因此需要定位到a標籤(tags)節點(elements)內的屬性(atttributes)"href"

In [ ]:
for tag in a_tags:
  # 輸出超連結網址
  print(((tag.get('href'))))

### 4.2.6 在找到超連結網址之後，用相同的方法，可以讓我們找到其他關於文章的資訊，而在此之前，可以先試著找出文章的區塊(class_='bpyTee)

In [ ]:
topic_block = soup.find_all('article', class_='tgn9uw-0 bReysV')
print(topic_block)

### 4.2.7 找到所有希望爬取的變數tag('標題','類別','學校','摘要','讚數','回應','連結')中的content

In [ ]:
df = pd.DataFrame(columns = ['標題','類別','學校','摘要','讚數','回應','連結'])

for i in topic_block:
    topic_name = i.find('a',class_='tgn9uw-3 cUGTXH').text
    category = i.find('div', class_ = 'euk31c-3 iVGtVg').text
    school = i.find('div',class_='euk31c-3 iVGtVg').next_sibling.text
    good = i.find('div', class_='cgoejl-3 jMiYgp').text
    exerpt = i.find('div', class_='tgn9uw-4 jtaqbD').text
    reply = i.find('div',class_='uj732l-2 ghvDya').span.text
    link = 'https://www.dcard.tw{}'.format(str(i.find('a', class_='tgn9uw-3 cUGTXH')['href']))
    df.loc[len(df)] = [topic_name,category,school,exerpt,good,reply,link]
df.head()

# 5. API實作

## 5.1 建立API的基本概念

API 是 Application Programming Interface的縮寫，中文是應用程式介面。把API比擬成實際上生活常用的例子，就如同我們在餐廳點餐，顧客(User)向服務生(API)點餐，並藉由服務生傳遞填寫完的菜單給廚房(Database)，最後在由廚房(Database)出餐給服務生(API)並交給顧客(User)。

也因此在爬蟲中使用API就如同我們透過API向網頁後端的database發出資料的請求(request)，若database有回應(response)的話會再次透過API回傳資料給我們，並呈現在我們的介面當中。

![Image Test](how_an_api_works.png 'API')


In [10]:
import cloudscraper

scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
# Or: scraper = cloudscraper.CloudScraper()  # CloudScraper inherits from requests.Session
print(scraper.get(url).text)  # => "<!DOCTYPE html><html><head>..."

{"id":234376747,"title":"#請益 極光水跟原生露請益","content":"小弟的膚質是混合偏乾 T字出油兩頰偏乾 大家覺得那個比較好 雖然我有拿過極光水試用 而且效果我還蠻滿意的 但因為快要冬天 所以很猶豫那個會比較適合 而且加上週年慶 如果買蘭蔻會順便帶一瓶小黑 雅詩的話會順便帶新版小棕 所以整個超級疑惑的啊 請板上的各位水水們為小弟解惑一下","excerpt":"小弟的膚質是混合偏乾 T字出油兩頰偏乾 大家覺得那個比較好 雖然我有拿過極光水試用 而且效果我還蠻滿意的 但因為快要冬天 所以很猶豫那個會比較適合 而且加上週年慶 如果買蘭蔻會順便帶一瓶小黑 雅詩的話","anonymousSchool":false,"anonymousDepartment":true,"pinned":false,"forumId":"cbd5285f-3cba-4bfc-86d0-1ab52d201459","replyId":null,"createdAt":"2020-09-08T10:13:29.940Z","updatedAt":"2020-09-08T10:13:29.940Z","commentCount":24,"likeCount":23,"tags":["HIDE_THUMBNAIL"],"topics":["原生露","極光水"],"supportedReactions":null,"withNickname":false,"reportReason":"","hiddenByAuthor":false,"meta":{"layout":"classic"},"forumName":"美妝","forumAlias":"makeup","nsfw":false,"school":"國立雲林科技大學","replyTitle":null,"gender":"M","personaSubscriptable":true,"reactions":[{"id":"286f599c-f86a-4932-82f0-f5a06f1eca03","count":23}],"hidden":false,"customStyle":null,"isSuspiciousAccount":false,"isModerator":false,"layout"

## 5.2 API實作

### 5.2.1 Dcard API基本設定

- What is Get Method in HTTP 
> GET requests are used to retrieve information about the resource specified by the URI. GET is a safe method — that means that a GET request should not result in any changes in the server state. It should not cause creation, updating or deletion of any application data. It should be used only for ‘read-only’ actions.


- Dcard API 中可使用的參數
 
|說明     |請求方式    |路徑    |
|:---|:--:|:------|
|全部文章	|GET	|/posts
|看板資訊	|GET	|/forums
|看板內文章列表	|GET	|/forums/{看板名稱}/posts
|文章內文	|GET	|/posts/{文章ID}
|文章內引用連結	|GET	|/posts/{文章ID}/links
|文章內留言	|GET	|/posts/{文章ID}/comments

In [ ]:
ID = '234376747'
url = 'https://www.dcard.tw/service/api/v2/posts/' + ID #設定API的網址

scraper = cloudscraper.create_scraper()  # 由於Dcard網頁有受到Cloudflare的保護，需要通過驗證才能使用，在這邊我們使用 cloudscraper 專門就是要拿來繞過 Cloudflare 頁面的套件，而它是建立在 Requests 之上
# Or: scraper = cloudscraper.CloudScraper()  # CloudScraper inherits from requests.Session
# print(scraper.get(url).text)  # => "<!DOCTYPE html><html><head>...
requ = scraper.get(url) #透過get method取得網頁的資訊
requ.json() #轉為Json檔

### 5.2.1 爬API原始可以爬的數量上限(100)篇文章

In [ ]:
#開爬
url = 'https://www.dcard.tw/service/api/v2/posts?popular=true&limit=100'
resq = scraper.get(url)
rejs = resq.json()
# for i in range(len(rejs)):
#     df = df.append(Crawl(rejs[i]['id']),ignore_index=True)
#     time.sleep(0.05)
# print(df.shape)
print(rejs)

### 5.2.2 設定我們需要的資料欄位，再把爬下來的資料從json轉為df

In [ ]:
df = pd.DataFrame(rejs, columns=['id','title','content','excerpt','createdAt','updatedAt','commentCount','forumName','forumAlias','gender','likeCount','reactions','topics'])
df

### 5.2.3 由於API一次只能爬一百篇文章，需要透過迴圈爬完想爬的文章數量

In [45]:
for j in range(5):
    last = str(int(df.tail(1).id)) # 找出原本資料中的最後一筆ID
    url = 'https://www.dcard.tw/service/api/v2/posts?popular=true&limit=100&before=' + last # 設定成新的網址
    resq = scraper.get(url) #發出request取得新的資訊
    rejs = resq.json()
    df = df.append(pd.DataFrame(rejs), ignore_index=True, sort = False)
    df

236432030
236428817
236432906
236431483
236433249


(600, 49)